In [1]:
import pandas as pd
import random as r
from sklearn.metrics import balanced_accuracy_score
from sklearn.neighbors import KNeighborsClassifier 
from imblearn.over_sampling import SMOTE

Using TensorFlow backend.


In [2]:
preprocessed=pd.read_csv("../PreprocessedDataset.csv")
habitableRows=list(preprocessed.rowid[preprocessed.habitable==True])

In [3]:
def prepareData():
    #List of columns to be used for training
    #it will be all columns in preprocessed except for "habitable" and "rowid"
    trainCols=[x for x in preprocessed.columns if x not in ['habitable','rowid']]
    validate=[]
    Hcopy=habitableRows.copy()
    #numHidden sets the number of habitable planets to use for validation of the model
    numHidden=round(len(Hcopy)/2)

    #Randomly select habitable exoplanets and 
    #add their row_id to validation set 
    print("Hiding ",numHidden," habitable(",end="")
    for i in range(numHidden): 
        randNum=r.randint(0,len(Hcopy)-1)  
        validate.append(Hcopy[randNum])
        print(Hcopy[randNum],end=",")
        del Hcopy[randNum]
    print("\b )")

    #Add row_id of non-habitable planets to the validation set till its length becomes 200
    while len(validate)<200:
        temp=r.randint(0,len(preprocessed)-1)
        if temp not in habitableRows and temp not in validate:
            validate.append(temp)

    #Take all columns of the planets whose row_id is in "validate" variable(in the validation set) and
    #store it in "validate" variable
    validate=preprocessed[preprocessed.rowid.isin(validate)]

    #Store the planets that are not in the validation set in the training set
    trainingSet=preprocessed[~preprocessed.rowid.isin(validate.rowid)]

    #Store the training features in X and target feature(habitable or not) in y 
    X=trainingSet[trainCols]
    y=trainingSet.habitable

    #the SMOTE library mutates existing data to creating more data
    #Here we use SMOTE to increase the number of habitable planets in the training and validation data
    smote = SMOTE(ratio='minority')
    X_sm, y_sm = smote.fit_sample(X, y)
    validateX,validateY=smote.fit_sample(validate[trainCols],validate.habitable)
    return X_sm,y_sm,validateX,validateY

# Effect of Random Seed on model
 Does changing the random seed during training but keeping the same dataset substantially change the outcome for this model?

In [4]:
RandomSeed=42
X_sm,y_sm,validateX,validateY=prepareData()
n_neighbours=[1,2,5,7]
algorithm=['auto','ball_tree','kd_tree','brute']
leaf_size=[10,20,30,40]
p=[1,2,5]
for testNumber in range(5):
    print("*"*10,end="\n\n")
    print("TEST NUMBER",testNumber+1,"Random Seed =",RandomSeed)
    r.seed(RandomSeed)
    RandomSeed=RandomSeed+10
    Bestscores=[0,0]
    for n in n_neighbours:
        for a in algorithm:
            for l in leaf_size:
                for P in p:
                    model=KNeighborsClassifier(n_neighbors=n,algorithm=a,leaf_size=l,p=P)
                    model.fit(X_sm,y_sm)
                    y_preds=model.predict(validateX)
                    currScore=balanced_accuracy_score(validateY,y_preds)
                    if(currScore>Bestscores[0]):
                        Bestscores[0]=currScore
                        Bestscores[1]=model.get_params()
    print("BEST SCORE:",str(Bestscores[0])+"\n"+"PARAMS:",Bestscores[1])

Hiding  24  habitable(2883,3716,3132,1604,2882,2129,2156,3741,3742,1845,151,1137,2155,2189,2316,1147,3115,1205,2014,130,1227,114,2547,2223, )
**********

TEST NUMBER 1 Random Seed = 42
BEST SCORE: 0.8039772727272727
PARAMS: {'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 7, 'p': 5, 'weights': 'uniform'}
**********

TEST NUMBER 2 Random Seed = 52
BEST SCORE: 0.8039772727272727
PARAMS: {'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 7, 'p': 5, 'weights': 'uniform'}
**********

TEST NUMBER 3 Random Seed = 62
BEST SCORE: 0.8039772727272727
PARAMS: {'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 7, 'p': 5, 'weights': 'uniform'}
**********

TEST NUMBER 4 Random Seed = 72
BEST SCORE: 0.8039772727272727
PARAMS: {'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'metric_params': Non

# Effect of changing random seed when preparing data on model

In [5]:
RandomSeed=42
n_neighbours=[1,2,5,7]
algorithm=['auto','ball_tree','kd_tree','brute']
leaf_size=[10,20,30,40]
p=[1,2,5]
for testNumber in range(5):
    print("*"*10,end="\n\n")
    print("TEST NUMBER",testNumber+1,"Random Seed =",RandomSeed)
    r.seed(RandomSeed)
    X_sm,y_sm,validateX,validateY=prepareData()
    RandomSeed=RandomSeed+10
    Bestscores=[0,0]
    for n in n_neighbours:
        for a in algorithm:
            for l in leaf_size:
                for P in p:
                    model=KNeighborsClassifier(n_neighbors=n,algorithm=a,leaf_size=l,p=P)
                    model.fit(X_sm,y_sm)
                    y_preds=model.predict(validateX)
                    currScore=balanced_accuracy_score(validateY,y_preds)
                    if(currScore>Bestscores[0]):
                        Bestscores[0]=currScore
                        Bestscores[1]=model.get_params()
    print("BEST SCORE:",str(Bestscores[0])+"\n"+"PARAMS:",Bestscores[1])

**********

TEST NUMBER 1 Random Seed = 42
Hiding  24  habitable(3233,163,117,2031,2014,1845,1137,703,3716,153,3922,2883,130,128,1205,2156,2223,151,2542,1604,3133,3115,3741,2880, )
BEST SCORE: 0.7869318181818181
PARAMS: {'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 7, 'p': 5, 'weights': 'uniform'}
**********

TEST NUMBER 2 Random Seed = 52
Hiding  24  habitable(2014,130,3962,2882,2547,2189,2503,128,1137,1227,2883,1604,2541,3233,114,152,3132,3742,117,1424,163,1205,1845,2542, )
BEST SCORE: 0.7471590909090909
PARAMS: {'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 7, 'p': 2, 'weights': 'uniform'}
**********

TEST NUMBER 3 Random Seed = 62
Hiding  24  habitable(2902,1147,151,2014,2547,2135,3962,2223,1205,3233,1424,1227,114,2156,1845,986,128,3744,2441,2541,117,3741,2129,3115, )
BEST SCORE: 0.7244318181818181
PARAMS: {'algorithm': 'auto', 'leaf_size': 1